In [37]:
import os
import sys
import open3d as o3d
import numpy as np
import copy
import glob
import json

In [38]:
#-------------------------------------------------------------------------------
def rotate_pcd(pcd ,rotation_theta=90, center_pcd=None):

    theta = np.radians(rotation_theta)

    if center_pcd is not None:
        min_x, min_y, min_z = center_pcd.get_min_bound()
        max_x, max_y, max_z = center_pcd.get_max_bound()

        center_x = abs(max_x - min_x)/2
        center_y = abs(max_y - min_y)/2
        center_z = abs(max_z - min_z)/2
    else:
        min_x, min_y, min_z = pcd.get_min_bound()
        max_x, max_y, max_z = pcd.get_max_bound()

        center_x = abs(max_x - min_x)/2
        center_y = abs(max_y - min_y)/2
        center_z = abs(max_z - min_z)/2

    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                [np.sin(theta), np.cos(theta), 0],
                [0, 0, 1]])

    rotated_pcd = pcd.rotate(rotation_matrix, center=[center_x, center_y, center_z])

    return rotated_pcd


#-------------------------------------------------------------------------------
def open_pointcloud(file_path):
    
    pcd = o3d.io.read_point_cloud(file_path,
                                        remove_nan_points=True,
                                        remove_infinite_points=True)
    
    down_pcd = copy.deepcopy(pcd).voxel_down_sample(voxel_size=15)

    return down_pcd


#-------------------------------------------------------------------------------
def load_metadata_dict(path):
    
    with open(path) as f:
        meta = json.load(f)['lemnatec_measurement_metadata']

    return meta


#-------------------------------------------------------------------------------

In [39]:
# Open the raw point cloud, directly from the gantry
down_east_pcd = open_pointcloud('./scanner3DTop-2023-07-21__12-59-17-030_sorghum/2023-07-21__16-13-54-334/ec637c9f-403f-4b71-8aaf-cac171121bd9__Top-heading-east_0.ply')
down_west_pcd = open_pointcloud('./scanner3DTop-2023-07-21__12-59-17-030_sorghum/2023-07-21__16-13-54-334/ec637c9f-403f-4b71-8aaf-cac171121bd9__Top-heading-west_0.ply')

# Open the metadata
metadata = load_metadata_dict('./scanner3DTop-2023-07-21__12-59-17-030_sorghum/2023-07-21__16-13-54-334/ec637c9f-403f-4b71-8aaf-cac171121bd9_metadata.json')

# Rotate point cloud
new_east_down = rotate_pcd(down_east_pcd,90)
new_west_down = rotate_pcd(down_west_pcd,90)

# Apply initial, required translation
if metadata['gantry_system_variable_metadata']['scanIsInPositiveDirection'] == "False":

    new_east_down = new_east_down.translate([0,(float(metadata['gantry_system_variable_metadata']['position x [m]'])-3.798989)/(8.904483-7.964989)*1000,0])
    new_west_down = new_west_down.translate([0,(float(metadata['gantry_system_variable_metadata']['position x [m]'])-3.798989)/(8.904483-7.964989)*1000,0])
    pcd_direction = "Negative"

else:

    new_east_down = new_east_down.translate([22280.82692587,(float(metadata['gantry_system_variable_metadata']['position x [m]'])-3.798989)/(8.904483-7.964989)*1000,0])
    new_west_down = new_west_down.translate([22280.82692587,(float(metadata['gantry_system_variable_metadata']['position x [m]'])-3.798989)/(8.904483-7.964989)*1000,0])
    pcd_direction = "Positive"

# Redefine the scan direction
scan_direction = 0.0 if pcd_direction=="Negative" else 1.0

# Apply the transformation to the point cloud
final_pcd = copy.deepcopy(new_east_down).transform(<transformation here>)

# Save the rotated, transformed point cloud
o3d.io.write_point_cloud("net_results.ply", final_pcd)

In [36]:
##################### IGNORE #####################
# neg = new_east_down.transform(np.load('2023-07-21_ew_negative_final_transformation.npy'))
# pos = new_east_down.transform(np.load('2023-07-21_ew_positive_final_transformation.npy'))
# o3d.io.write_point_cloud("positive.ply", pos)
# o3d.io.write_point_cloud("negative.ply", neg)

# if scan_direction == 1.0:
#     new_east_down.transform(<transformation>)
# else:
#     new_west_down.transform(<transformation>)

True